In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/Poco_new.csv')

In [3]:
df.duplicated().sum()

4

In [4]:
df_P= df.drop_duplicates()

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
df_P['Name']=df_P['Company Name'].str.extract('(POCO)')

In [7]:
df_P['test'] = df_P['Company Name'].str.extract('(\([^)]*\,)')
df_P['test1']=df_P['test'].astype(str)
df_P['Color'] = df_P['test1'].apply(lambda x: x.replace('(','').replace(',',''))

In [8]:
df_P['test2'] = df_P['Company Name'].str.split('(').str[0]
df_P['Model']= df_P['test2'].str.split().str[1:].str.join(' ')

In [9]:
def match(x):
    if re.search("Ratings",x):
        return x
    else:
        return "0"

In [10]:
df_P['test3']=df_P['Ratings'].apply(match)
df_P['test5']=df_P['test3'].apply(lambda x: x[x.find("SAMSUNG") : x.find("Ratings")])
df_P['Ratings(*)']=df_P['test3'].str.extract('(4\.\d|3\.\d)')
df_P['Ratings(*)']=df_P['Ratings(*)'].fillna(0)

In [11]:
df_P['test4']=df_P['test3'].apply(lambda x: x[x.find("Ratings") : x.find("Reviews")])
df_P['Reviews']=df_P['test4'].apply(lambda x: x.replace('Ratings','').replace('&','').replace(',',''))
df_P['Reviews']=df_P['Reviews'].str.strip()

In [12]:
def remove_whitespace(s):
    return ' '.join(s.split())

In [13]:
df_P['MobilesSold']=df_P['Reviews'].apply(remove_whitespace)
df_P['MobilesSold']=df_P['MobilesSold'].replace('','0')
df_P['MobilesSold']=df_P['MobilesSold'].astype(int)

In [14]:
df_P['test6']=df_P['Mobile Price'].apply(lambda x: x.replace('%','').replace('offFree','').replace('delivery','').replace('off','').replace('Free delivery','').replace('Free',''))

In [15]:
df_P['OfferInPerentage'] =df_P['offer'].str.extract('(\d+)')
df_P['OfferInPerentage']=df_P['OfferInPerentage'].fillna(0)
df_P['OfferInPerentage']=df_P['OfferInPerentage'].astype(int)

In [16]:
df_P['test7']=df_P['test6'].str.extract('(\₹[^)]*\₹)')

In [17]:
df_P['test7'] = np.where(df_P['test7'].isnull(),df_P['test6'],df_P['test7'])

In [18]:
df_P['DiscountedPrice']=df_P['test7'].apply(lambda x:str(x).replace('₹','').replace(',','').replace('Price: Not Available','nan'))
#Replacing NAN Values from Price(with discount) column
df_P['DiscountedPrice']=df_P['DiscountedPrice'].replace('nan','0')
df_P['DiscountedPrice']=df_P['DiscountedPrice'].astype(int)

In [19]:
df_P['MaxRetailPrice'] = (df_P.DiscountedPrice + ((df_P.OfferInPerentage/100) * df_P.DiscountedPrice))
df_P['MaxRetailPrice']=df_P['MaxRetailPrice'].round(0)

In [20]:
df_P['Sales(₹ CR)']=(df_P.DiscountedPrice * df_P.MobilesSold )/10000000
df_P['Sales(₹ CR)']=df_P['Sales(₹ CR)'].round(2)

In [21]:
df_P['test8']= df_P['Specifications'].str.split('|'and'ROM').str[0]
df_P[['RAM(GB)','ROM(GB)']]= df_P['test8'].str.split('|',expand = True) 
df_P['RAM(GB)']=df_P['RAM(GB)'].apply(lambda x:str(x).replace('GB','').replace('RAM',''))
df_P['ROM(GB)']=df_P['ROM(GB)'].str.replace('GB','')

In [22]:
df_P['Display']= df_P['Specifications'].str.extract('\((.*?)\)')
df_P['Display']=df_P['Display'].str.replace('inch','')

In [23]:
df_P['Screen']=df_P['Specifications'].str.extract('(Quad HD+|HD+|Full HD+ AMOLED|QGVA|AMOLED)')
df_P['Screen']=df_P['Screen'].fillna("Not Mentioned")

In [24]:
df_P['test9']= df_P['Specifications'].apply(lambda x: x[x.find("Display"):x.find("Front")])

In [25]:
df_P['test9']=df_P['test9'].apply(lambda x:x[x.find("Display"):x.find("Dual")])
df_P['test9']=df_P['test9'].apply(lambda x:x[x.find("Display"):x.find("Rear")])

In [26]:
df_P['test10'] = df_P['test9'].apply(lambda x:str(x).replace('Display','').replace('+',''))

In [27]:
df_P[['Rear Camera','Front Camera']]= df_P['test10'].str.split('|',expand = True)

In [28]:
df_P['No of Rear Camera']= df_P['Rear Camera'].str.count("MP")
df_P['No of Front Camera']= df_P['Front Camera'].str.count("MP")
df_P['No of Front Camera']= df_P['No of Front Camera'].fillna(0)

In [29]:
df_P['Battery']= df_P['Specifications'].str.extract('(\d{4} mAh)')
df_P['Battery']=df_P['Battery'].apply(lambda x: str(x).replace('mAh',''))

In [30]:
df_P['Processor']= df_P['Specifications'].str.extract('(Exynos|Mediatek|Snapdragon|Octa Core|208MHz|8 Gen 1|UMS9230|quad core|Helio)')
df_P['Processor']=df_P['Processor'].fillna("Not Mentioned")

In [31]:
# drop all test columns and unwanted columns
df_P.drop(columns=['test','test1','test2','test3','test4','test5','test6','test7','test8','test9','test10','Company Name','Ratings','Specifications','Mobile Price','offer'],inplace=True,axis=1)

In [32]:
df_P.to_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/POCOCleaned.csv',index=False)